<a href="https://colab.research.google.com/github/sindhura-nk/Artificial-Intelligence-1446/blob/main/Next_sentence_Word_Generation_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Buisness Scenario: Build a model that can predict the next words when some words are provided to it.

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
# Import all the necessary libraries
from warnings import filterwarnings
filterwarnings(action='ignore')

# Preprocess the text
from tensorflow.keras.preprocessing.text import Tokenizer # creates work tokens, number sequences
from keras.preprocessing import sequence # padding
# Model
from keras.models import Sequential
from keras.layers import Input,Dense,LSTM,Embedding

## LOad the dataset

In [ ]:
with open("/content/Harry Potter and the Sorcerer's Sto.txt",'r') as file:
  data = file.read()
print(data[:50])

In [ ]:
data[:50]

We are using Keras Tokenizer here. Few points are discussed below related to fit_on_texts method and texts_to_sequences method:
tokenizer.fit_on_texts(texts) : Builds the word index (vocabulary) from your dataset.

It looks at the list of texts you give it. It assigns a unique integer index to each unique word.
The most frequent word gets index 1, the next most frequent 2, and so on. Index 0 is reserved for padding.

Example: texts = ["The wizard cast a spell", "The spell was powerful"]
tokenizer.fit_on_texts(texts)
print(tokenizer.word_index)
Output: {'the': 1, 'spell': 2, 'wizard': 3, 'cast': 4, 'a': 5, 'was': 6, 'powerful': 7}
tokenizer.texts_to_sequences(texts): Converts your texts into sequences of integers based on the word index created by fit_on_texts.

Example: sequences = tokenizer.texts_to_sequences(["The wizard cast a spell"])
print(sequences)
Output: [[1, 3, 4, 5, 2]]

In [ ]:
# Intializing the tokenizer
tokenizer = Tokenizer()

# apply the tokenizer on respective words
texts = ["The wizard cast a spell", "The spell was powerful"]
tokenizer.fit_on_texts(texts) # this is going to develop tokens and their frequency count
# the : 2
# wizard: 1
# cast: 1
# a : 1
# spell:2
print(tokenizer.word_index)
# word indexing: the:1, spell:2, wizard:3, cast:4, a:5.....


In [ ]:
# if we want to give this text to the model
# [[1,3,4,5,2],[1,2,6,7]]
tokenizer.texts_to_sequences(texts)

In [ ]:
# Intializing the tokenizer
tokenizer = Tokenizer()

# Fit on texts - pass the data
tokenizer.fit_on_texts([data])
word_ind = tokenizer.word_index
word_ind

In [ ]:
reverse_word_index = {value:key for key,value in word_ind.items()}
reverse_word_index

## Text on sequences

In [ ]:
total_length = len(word_ind) + 1
# to reserve for padding

In [ ]:
total_length

In [ ]:
input_sequences = []
for line in data.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  # [4,5,6,7]
  for i in range(1,len(token_list)): # for i in range(1,4)
    n_gram_sequence = token_list[:i+1] # token_list[0:2] = 0th index,1st index
    # token_list[0:3] = 0th index,1st index,2nd index
    # token_list[0:4] = 0th index, 1st index, 2nd index, 3rd index
    input_sequences.append(n_gram_sequence) # [[4,5],[4,5,6],[4,5,6,7]]
print(input_sequences[:15])

## padding to bring all the ngram tokens to same size. to bring all the setences to same length

In [ ]:
max([90,89,77])

In [14]:
max_length = max([len(lines) for lines in input_sequences])

input_sequences = sequence.pad_sequences(input_sequences,maxlen=max_length)
input_sequences[:15]

array([[  0,   0,   0, ...,   0,   7, 121],
       [  0,   0,   0, ...,   7, 121,   2],
       [  0,   0,   0, ..., 121,   2,   1],
       ...,
       [  0,   0,   0, ..., 253, 220,   6],
       [  0,   0,   0, ..., 220,   6, 612],
       [  0,   0,   0, ...,   6, 612, 316]], dtype=int32)

In [15]:
print(max_length)

202


## Separate x and y

In [16]:
x = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [17]:
x[:10]

array([[  0,   0,   0, ...,   0,   0,   7],
       [  0,   0,   0, ...,   0,   7, 121],
       [  0,   0,   0, ...,   7, 121,   2],
       ...,
       [  0,   0,   0, ...,   0,   1, 144],
       [  0,   0,   0, ...,   1, 144,  63],
       [  0,   0,   0, ...,   0,   0, 150]], dtype=int32)

In [18]:
y[:10]

array([ 121,    2,    1,  634,  158,   38,  144,   63, 1049,    2],
      dtype=int32)

In [19]:
y.shape

(74892,)

If you want to use categorical_crossentropy, then convert y into y cateogrical.
    y_cat = to_categorical(y)

## Model Buidling

In [20]:
model = Sequential()
# Provide the input as max length indicating total number of sentences
model.add(Input((max_length,)))
# Add the layers
model.add(Embedding(input_dim=total_length,output_dim=300,trainable=False))
model.add(LSTM(200,return_sequences=True,dropout=0.3)) # return sequences will provide the sequences to next LSTM layer
model.add(LSTM(150,dropout=0.2))
# Add one Hidden layer
model.add(Dense(100, activation='tanh'))
# Add output layer
model.add(Dense(total_length, activation='softmax'))


In [21]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='accuracy',patience=5)

In [ ]:
nn = model.fit(x,y,validation_split=0.2,epochs=10,callbacks=[early_stop])

Epoch 1/10
1382/1873 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.0448 - loss: 7.0611

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
def generate_text(user_text, next_words=10):
    for _ in range(next_words):
        # preparing our user text ready for the model
        token_list = [tokenizer.word_ind.get(w, 0) for w in word_tokenize(user_text.lower())]
        token_list = sequence.pad_sequences([token_list],maxlen=max_length-1)
        # give the proceesed text to model for prediction of next 50words
        predicted_probs = model.predict(token_list.reshape(1, max_length-1), verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1) # it gives you the index of next word
        output_word = reverse_word_index.get(predicted[0], '') # convert the index into word using index_word
        user_text += ' ' + output_word
        # user_text = user_text + output_word
    return user_text

In [ ]:
print(generate_text("The Wizard is"))